[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pdf-tools/components-code-sample-hub/blob/main/jupyter/pdftools_sdk/pdftools_sdk_decrypt.ipynb)

In [ ]:
%pip install https://pdftools-public-downloads-production.s3.eu-west-1.amazonaws.com/productkits/PDFSDK/latest/pdftools_sdk-latest.tar.gz
%pip install ipython

# Decrypt an encrypted PDF
Remove encryption from a PDF.

In [ ]:
import io
from pdftools_sdk.pdf import Document
from pdftools_sdk.sign import OutputOptions as SignOutputOptions, SignatureRemoval, Signer

In [ ]:
# Download a file from a given URL and save it to the local system
def prepare_file(url: str, path: str):
    import requests
    response = requests.get(url)
    response.raise_for_status()

    with open(path, 'wb') as f:
        f.write(response.content)

In [ ]:
# Set input arguments
password = 'pdftools'
input_url = 'https://pdftools-public-downloads-production.s3.eu-west-1.amazonaws.com/samples/testfiles/InvoiceEncrypted(pwd_pdftools).pdf'
input_path = 'InvoiceEncrypted(pwd_pdftools).pdf'
prepare_file(input_url, input_path)
output_path = 'Decrypted.pdf'

In [ ]:
def decrypt(password, input_path, output_path):
    # Use password to open encrypted input document
    with io.FileIO(input_path, 'rb') as in_stream:
        with Document.open(in_stream, password) as input_document:
            if input_document.permissions == None:
                print(f"Input file is not encrypted.")
                return

            # Create stream for output file
            with io.FileIO(output_path, 'wb+') as output_stream:
                # Set encryption options
                output_options = SignOutputOptions()
                # Set encryption parameters to no encryption
                output_options.encryption = None
                # Allow removal of signatures. Otherwise the Encryption property is ignored for signed input documents
                # (see warning category Sign.WarningCategory.SignedDocEncryptionUnchanged).
                output_options.remove_signatures = SignatureRemoval.SIGNED

                # Decrypt the document
                signer = Signer()
                signer.process(input_document, output_stream, output_options)

In [ ]:
try:
    # By default, a test license key is active. In this case, a watermark is added to the output. 
    # If you have a license key, please uncomment the following call and set the license key.
    # from pdftools_sdk.sdk import Sdk
    # Sdk.initialize("<PDFSDK,V1,MGAASQD6L2JMQHL54PK08RQX8GG4SS0M8DAHVPH0VMP3NB8R9DUK>")

    # Decrypt a PDF document
    decrypt(password, input_path, output_path)

    print(f"Successfully created decrypted {output_path}")
except Exception as e:
    print(f"An error occurred: {e}")